# Reading files

Standard Python has a quite powerful mechanism for reading files. To open a file you can use the built-in `open`-function as follows:

    with open('filename.txt', mode='r') as fid:
        ...
        

This will open the file names `filename.txt` for reading (`mode='r'`). A handle of the file will be stored in the `fid`-variable. The `with`-construct is strictly speaking not necessary, but highly recommended. Using `with` handles closing of the file for us in a proper manner, for instance making sure the file is closed even if some exception occurs during reading.

## Reading lines one at a time

The following is a concrete example of how to open a file and do some simple operations on it. In this case we only print headlines from the `README.md` file.

In [ ]:
with open('README.md', mode='r') as fid:
    for line in fid:
        if line.startswith('#'):
            print(line.lstrip('#').strip())

The file handle (stored in `fid`) can be iterated over. The `for`-loop returns the lines of the file one at a time. The `README.md`-file is in the Markdown-format where headlines are marked with the hash-symbol (`#`). To find the headlines we therefore test if a given line starts with `#`, and if it does we print it out. Finally, when printing we strip off the hash-symbols (`lstrip('#')` removes `#` on the `l`eft side of the string) and whitespace (`strip()` without arguments removes whitespace like spaces, tabs and newlines on both sides of the given string).

## Reading the whole file at once

It is also easy to read in the whole file at once, postponing any processing until later. The following example eventually does the same as the one above, although using a somewhat different technique.

In [ ]:
with open('README.md', mode='r') as fid:
    lines = fid.readlines()

headlines = [line.lstrip('#').strip()
             for line in lines
             if line.startswith('#')]
print('\n'.join(headlines))

The `readlines()`-method reads all lines from the file and stores them in a list that we process later. Note that the `with`-context ends after `readlines()` is called. This means that the file is closed at this moment. To confirm that the whole file is read, we can inspect the `lines`-list.

In [ ]:
lines

To find the `headlines` we filter `lines` using a _list comprehension_. A list comprehension typically takes a list and creates another list. It consists of three parts. The middle part, `for line in lines`, looks like a regular `for`-loop. The first part describes each element in the new list. In this case we strip off hash-symbols and whitespace as in the previous example. The third part, `if line.startswith('#')` is optional, but can be used to filter which elements of the list to keep. In this case we keep the lines that starts with the hash-symbol.

`headlines` is now a list of strings (see below). To print out the headlines, one on each line, we join the list of strings using a newline (`\n`) between each string.

In [ ]:
headlines

## The real world is usually more dirty ...

Data encountered in the real world often need a lot of cleaning before they can be used. We will here have a look at a real data format, and try to read the data in a form that is usable.

In [ ]:
!cat -n data/dirty_data.txt

This data file contains one header line while each of the following lines describes a measurement. Each line consists of several fields separated by whitespace. Each field has the form `<key>+<value>` where the value is padded by `0`s. The first field is an identifier, while the rest of the fields (except the last one, which can be ignored) are different measurements.

We want to read out the measurements for each identifier.

### Parsing a field of the form `<key>+<value>`

Let us start with making sense of each field. They are all on the form `<key>+<value>`. These can be separated using either `split` or `partition`. The latter is quite nice to use as long as we know that there are two and only two parts, while `split` works great if there can be more than two parts.

We have already seen `strip` which can be used to strip away the leading `0`s of each `value`. The following function accepts a field as a string and returns `value` as a string (we'll simply throw away the `key` for now):

In [ ]:
def strip_field(field):
    key, plus, value = field.partition('+')
    return value.lstrip('0')

Let's test it:

In [ ]:
print(strip_field('*410001+0000000000000021'))
print(strip_field('*110002+0000000000000CP4'))
print(strip_field('21.322+0000000044.98320'))

### Reading many fields

Next, we need to read in the data file, using the above function to strip each field in the process.

We'll start with simply throwing away the header. The call to `next` gives us the next value of the file handle (in practice the next line). It does return the current value of `fid` which we could have (and will use later) used to store the information in the header. Then we will store the fields of each line in a list.

In [ ]:
lines = list()

with open('data/dirty_data.txt', mode='r') as fid:
    next(fid)   # Throw away header line
    for line in fid:
        fields = [strip_field(f) for f in line.strip().split()]
        lines.append(fields)
        
lines

### Collecting the measurements for each identifier

Now that we have the data in a more readable form, we see that there are many measurements for each identifier (actually there are 5 series of measurements and for each series we measure each identifier twice, the order is `CP4`, `GRAV_RA`, `CP6`, `CP5` first forward, and then backward).

For the processing we want to collect all the measurements for each identifier. The order of the identifiers will be important later, so we'll use an `OrderedDict` for storage to make sure we keep the order.

---

Note that in Python 3.6 also regular `dict` will keep the order. However, this is not true for older Pythons and it is not guaranteed to be true for future Pythons.

In [ ]:
from collections import OrderedDict
data = OrderedDict()

for line in lines:
    name = line[0]   # First column is the identifier
    values = [float(v) for v in line[1:-1]]   # Convert to float, ignoring the first and last column
    data.setdefault(name, list()).append(values)   # setdefault creates the list if it does not already exist
    
data

We will leave the data like this for now. The list-of-lists is not an ideal data structure for doing data analysis. Therefore, we will return to this example in the `numpy`-notebook, and see how the `numpy.arrays` are much more convenient when working with data.

However, we can still tease out some numbers and do calculations on them. The following example gets the values of the first column for the `CP4`-data, and calculates their mean:

In [ ]:
data['CP4']

In [ ]:
values = [row[0] for row in data['CP4']]
values

In [ ]:
sum(values) / len(values)